In [19]:
import pandas as pd


In [20]:
df = pd.read_csv("reddit_data.csv")


In [21]:
df.head()


,comment_id,post_id,subreddit,author_username,comment_body,comment_score,comment_created_utc,parent_id,is_top_level,post_title,post_text,post_created_utc,post_url
0,e5rlkl4,9etcsl,AskWomen,Reese1993,Some of the worst places you don’t realize unt...,131,2018-09-11T08:52:27,t3_9etcsl,1,What are some subtle signs of a toxic workplace?,NaN,2018-09-11T07:04:12,https://www.reddit.com/r/AskWomen/comments/9et...
1,e5rn12k,9etcsl,AskWomen,Niasmomma99,When you feel perfectly well until you pull in...,123,2018-09-11T09:21:00,t3_9etcsl,1,What are some subtle signs of a toxic workplace?,NaN,2018-09-11T07:04:12,https://www.reddit.com/r/AskWomen/comments/9et...
2,e5rhrre,9etcsl,AskWomen,crazynekosama,High turnover rate. If people aren't staying l...,318,2018-09-11T07:48:11,t3_9etcsl,1,What are some subtle signs of a toxic workplace?,NaN,2018-09-11T07:04:12,https://www.reddit.com/r/AskWomen/comments/9et...
3,e5rf6iy,9etcsl,AskWomen,msstark,People talk about each other behind their backs.,167,2018-09-11T07:07:42,t3_9etcsl,1,What are some subtle signs of a toxic workplace?,NaN,2018-09-11T07:04:12,https://www.reddit.com/r/AskWomen/comments/9et...
4,e5rf7y0,9etcsl,AskWomen,a-little-sleepy,If you come to work in a good mood and someone...,168,2018-09-11T07:08:20,t3_9etcsl,1,What are some subtle signs of a toxic workplace?,NaN,2018-09-11T07:04:12,https://www.reddit.com/r/AskWomen/comments/9et...


In [22]:
df.head(10)


,comment_id,post_id,subreddit,author_username,comment_body,comment_score,comment_created_utc,parent_id,is_top_level,post_title,post_text,post_created_utc,post_url
0,e5rlkl4,9etcsl,AskWomen,Reese1993,Some of the worst places you don’t realize unt...,131,2018-09-11T08:52:27,t3_9etcsl,1,What are some subtle signs of a toxic workplace?,NaN,2018-09-11T07:04:12,https://www.reddit.com/r/AskWomen/comments/9et...
1,e5rn12k,9etcsl,AskWomen,Niasmomma99,When you feel perfectly well until you pull in...,123,2018-09-11T09:21:00,t3_9etcsl,1,What are some subtle signs of a toxic workplace?,NaN,2018-09-11T07:04:12,https://www.reddit.com/r/AskWomen/comments/9et...
2,e5rhrre,9etcsl,AskWomen,crazynekosama,High turnover rate. If people aren't staying l...,318,2018-09-11T07:48:11,t3_9etcsl,1,What are some subtle signs of a toxic workplace?,NaN,2018-09-11T07:04:12,https://www.reddit.com/r/AskWomen/comments/9et...
3,e5rf6iy,9etcsl,AskWomen,msstark,People talk about each other behind their backs.,167,2018-09-11T07:07:42,t3_9etcsl,1,What are some subtle signs of a toxic workplace?,NaN,2018-09-11T07:04:12,https://www.reddit.com/r/AskWomen/comments/9et...
4,e5rf7y0,9etcsl,AskWomen,a-little-sleepy,If you come to work in a good mood and someone...,168,2018-09-11T07:08:20,t3_9etcsl,1,What are some subtle signs of a toxic workplace?,NaN,2018-09-11T07:04:12,https://www.reddit.com/r/AskWomen/comments/9et...
5,e5rh1na,9etcsl,AskWomen,ah04eo,"Gossip, exclusion, managers pitting employees ...",153,2018-09-11T07:36:19,t3_9etcsl,1,What are some subtle signs of a toxic workplace?,NaN,2018-09-11T07:04:12,https://www.reddit.com/r/AskWomen/comments/9et...
6,e5rpuwl,9etcsl,AskWomen,[deleted],The boss insisting that the workplace is like ...,183,2018-09-11T10:22:33,t3_9etcsl,1,What are some subtle signs of a toxic workplace?,NaN,2018-09-11T07:04:12,https://www.reddit.com/r/AskWomen/comments/9et...
7,e5rfau4,9etcsl,AskWomen,aveclesmuguets,Gossip. Which bosses do not care to take care ...,62,2018-09-11T07:09:35,t3_9etcsl,1,What are some subtle signs of a toxic workplace?,NaN,2018-09-11T07:04:12,https://www.reddit.com/r/AskWomen/comments/9et...
8,e5rg0e8,9etcsl,AskWomen,blerrycat,Boss talks politics like everyone agrees.,49,2018-09-11T07:20:16,t3_9etcsl,1,What are some subtle signs of a toxic workplace?,NaN,2018-09-11T07:04:12,https://www.reddit.com/r/AskWomen/comments/9et...
9,e5rogpg,9etcsl,AskWomen,[deleted],Miscommunication between everyone in the workp...,81,2018-09-11T09:51:09,t3_9etcsl,1,What are some subtle signs of a toxic workplace?,NaN,2018-09-11T07:04:12,https://www.reddit.com/r/AskWomen/comments/9et...


In [23]:
df.shape


(25148, 13)

In [10]:
df.columns


Index(['comment_id', 'post_id', 'subreddit', 'author_username', 'comment_body',
       'comment_score', 'comment_created_utc', 'parent_id', 'is_top_level',
       'post_title', 'post_text', 'post_created_utc', 'post_url'],
      dtype='object')

In [24]:
list(df.columns)


['comment_id',
 'post_id',
 'subreddit',
 'author_username',
 'comment_body',
 'comment_score',
 'comment_created_utc',
 'parent_id',
 'is_top_level',
 'post_title',
 'post_text',
 'post_created_utc',
 'post_url']

In [25]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25148 entries, 0 to 25147
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   comment_id           25148 non-null  object
 1   post_id              25148 non-null  object
 2   subreddit            25148 non-null  object
 3   author_username      25148 non-null  object
 4   comment_body         25148 non-null  object
 5   comment_score        25148 non-null  int64 
 6   comment_created_utc  25148 non-null  object
 7   parent_id            25148 non-null  object
 8   is_top_level         25148 non-null  int64 
 9   post_title           25148 non-null  object
 10  post_text            17705 non-null  object
 11  post_created_utc     25148 non-null  object
 12  post_url             25148 non-null  object
dtypes: int64(2), object(11)
memory usage: 2.5+ MB


In [27]:
df.describe()


,comment_score,is_top_level
count,25148.000000,25148.000000
mean,17.564299,0.660967
std,157.905318,0.473390
min,-229.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,1.000000
75%,3.000000,1.000000
max,9049.000000,1.000000


In [28]:
df.describe(include='all')


,comment_id,post_id,subreddit,author_username,comment_body,comment_score,comment_created_utc,parent_id,is_top_level,post_title,post_text,post_created_utc,post_url
count,25148,25148,25148,25148,25148,25148.000000,25148,25148,25148.000000,25148,17705,25148,25148
unique,25148,235,2,15721,24790,NaN,24924,5616,NaN,232,106,235,235
top,e5rlkl4,ovt9k9,AskMen,[deleted],This comment or post has been removed for dera...,NaN,2021-08-02T04:39:33,t3_ovt9k9,NaN,What has made you realize you were getting old?,I (25M) recently took my little sister (14) ou...,2021-08-01T19:38:14,https://www.reddit.com/r/AskMen/comments/ovt9k...
freq,1,2500,16812,3847,28,NaN,3,2500,NaN,2500,2500,2500,2500
mean,NaN,NaN,NaN,NaN,NaN,17.564299,NaN,NaN,0.660967,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,157.905318,NaN,NaN,0.473390,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,-229.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN


In [29]:
df.isnull().sum()


comment_id                0
post_id                   0
subreddit                 0
author_username           0
comment_body              0
comment_score             0
comment_created_utc       0
parent_id                 0
is_top_level              0
post_title                0
post_text              7443
post_created_utc          0
post_url                  0
dtype: int64

In [30]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [31]:
"""
SENTIMENT ANALYSIS FOR REDDIT DATA
"""

import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns

print("="*70)
print("REDDIT SENTIMENT ANALYSIS")
print("="*70)

# ============================================================================
# STEP 1: LOAD YOUR DATA
# ============================================================================

print("\n Loading Reddit data...")


df = pd.read_csv('reddit_data.csv')

print(f" Loaded {len(df):,} comments")
print(f"\nColumns in your data:")
print(df.columns.tolist())

# ============================================================================
# STEP 2: INITIALIZE SENTIMENT ANALYZER
# ============================================================================

print("\n Initializing sentiment analyzer...")
analyzer = SentimentIntensityAnalyzer()
print(" Ready to analyze!")

# ============================================================================
# STEP 3: ANALYZE SENTIMENT FOR EACH COMMENT
# ============================================================================

print(f"\n Analyzing {len(df):,} comments...")
print("This may take a few minutes for large datasets...")

results = []
for idx, row in df.iterrows():
    # Get the comment text
    comment_text = row['comment_body']
    
    # Handle empty or missing comments
    if pd.isna(comment_text) or str(comment_text).strip() == "":
        sentiment_score = 0.0
        sentiment_label = 'Neutral'
    else:
        # Analyze sentiment
        scores = analyzer.polarity_scores(str(comment_text))
        sentiment_score = scores['compound']
        
        # Classify
        if sentiment_score >= 0.05:
            sentiment_label = 'Positive'
        elif sentiment_score <= -0.05:
            sentiment_label = 'Negative'
        else:
            sentiment_label = 'Neutral'
    
    results.append({
        'sentiment_score': sentiment_score,
        'sentiment_label': sentiment_label
    })
    
    # Progress indicator
    if (idx + 1) % 5000 == 0:
        print(f"  Processed {idx + 1:,}/{len(df):,} comments...")

print(f"✓ Completed analyzing {len(df):,} comments!")

# ============================================================================
# STEP 4: ADD RESULTS TO DATAFRAME
# ============================================================================

print("\n Adding sentiment results to dataframe...")

# Convert results to dataframe
sentiment_df = pd.DataFrame(results)

# Add to original dataframe
df = pd.concat([df, sentiment_df], axis=1)

print(" Sentiment scores added!")

# ============================================================================
# STEP 5: SHOW RESULTS SUMMARY
# ============================================================================

print("\n" + "="*70)
print("RESULTS SUMMARY")
print("="*70)

print("\n Sentiment Distribution:")
print(df['sentiment_label'].value_counts())

print("\n Sentiment Percentages:")
sentiment_pct = df['sentiment_label'].value_counts(normalize=True) * 100
for label, pct in sentiment_pct.items():
    print(f"  {label}: {pct:.1f}%")

print(f"\n Average Sentiment Score: {df['sentiment_score'].mean():.3f}")
print(f"   Standard Deviation: {df['sentiment_score'].std():.3f}")

# ============================================================================
# STEP 6: SENTIMENT BY SUBREDDIT
# ============================================================================

print("\n" + "="*70)
print("SENTIMENT BY SUBREDDIT")
print("="*70)

subreddit_stats = df.groupby('subreddit').agg({
    'sentiment_score': 'mean',
    'comment_id': 'count',
    'sentiment_label': lambda x: x.value_counts().to_dict()
}).round(3)

subreddit_stats.columns = ['Avg Sentiment', 'Comment Count', 'Label Distribution']

print("\n", subreddit_stats)

# ============================================================================
# STEP 7: SHOW EXAMPLES
# ============================================================================

print("\n" + "="*70)
print("EXAMPLES")
print("="*70)

print("\n Most Positive Comments:")
top_positive = df.nlargest(5, 'sentiment_score')
for idx, row in top_positive.iterrows():
    print(f"\n  Score: {row['sentiment_score']:.3f}")
    print(f"  Subreddit: {row['subreddit']}")
    print(f"  Comment: {str(row['comment_body'])[:150]}...")

print("\n\n Most Negative Comments:")
top_negative = df.nsmallest(5, 'sentiment_score')
for idx, row in top_negative.iterrows():
    print(f"\n  Score: {row['sentiment_score']:.3f}")
    print(f"  Subreddit: {row['subreddit']}")
    print(f"  Comment: {str(row['comment_body'])[:150]}...")

# ============================================================================
# STEP 8: CREATE VISUALIZATIONS
# ============================================================================

print("\n" + "="*70)
print("CREATING VISUALIZATIONS")
print("="*70)

# Set style
sns.set_style("whitegrid")

# Create figure with 3 subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Reddit Sentiment Analysis Results', fontsize=16, fontweight='bold')

# Chart 1: Sentiment Distribution (Bar Chart)
sentiment_counts = df['sentiment_label'].value_counts()
colors = {'Positive': '#2ecc71', 'Neutral': '#95a5a6', 'Negative': '#e74c3c'}
bar_colors = [colors.get(label, '#3498db') for label in sentiment_counts.index]

axes[0].bar(sentiment_counts.index, sentiment_counts.values, color=bar_colors)
axes[0].set_title('Sentiment Distribution', fontsize=14)
axes[0].set_ylabel('Number of Comments', fontsize=12)
axes[0].set_xlabel('Sentiment', fontsize=12)

# Add count labels on bars
for i, v in enumerate(sentiment_counts.values):
    axes[0].text(i, v, f'{v:,}\n({v/len(df)*100:.1f}%)', 
                ha='center', va='bottom', fontsize=10, fontweight='bold')

# Chart 2: Sentiment Score Distribution (Histogram)
axes[1].hist(df['sentiment_score'], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[1].axvline(df['sentiment_score'].mean(), color='red', linestyle='--', 
               linewidth=2, label=f'Mean: {df["sentiment_score"].mean():.3f}')
axes[1].set_title('Sentiment Score Distribution', fontsize=14)
axes[1].set_xlabel('Sentiment Score', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].legend()
axes[1].grid(alpha=0.3)

# Chart 3: Sentiment by Subreddit
subreddit_sentiment = df.groupby('subreddit')['sentiment_score'].mean().sort_values()
colors_sub = ['#e74c3c' if x < -0.05 else '#2ecc71' if x > 0.05 else '#95a5a6' 
              for x in subreddit_sentiment.values]

axes[2].barh(subreddit_sentiment.index, subreddit_sentiment.values, color=colors_sub)
axes[2].set_title('Average Sentiment by Subreddit', fontsize=14)
axes[2].set_xlabel('Average Sentiment Score', fontsize=12)
axes[2].axvline(0, color='black', linestyle='-', linewidth=0.8)
axes[2].grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('reddit_sentiment_analysis.png', dpi=300, bbox_inches='tight')
print(" Saved visualization: reddit_sentiment_analysis.png")
plt.close()

# ============================================================================
# STEP 9: SAVE RESULTS TO CSV
# ============================================================================

print("\n" + "="*70)
print("SAVING RESULTS")
print("="*70)

output_file = 'reddit_data_with_sentiment.csv'
df.to_csv(output_file, index=False)

print(f" Saved results to: {output_file}")
print(f"  Total rows: {len(df):,}")
print(f"  Total columns: {len(df.columns)}")
print(f"\n New columns added:")
print(f"  - sentiment_score (numerical: -1 to +1)")
print(f"  - sentiment_label (categorical: Positive/Neutral/Negative)")

# ============================================================================
# STEP 10: ADDITIONAL ANALYSIS
# ============================================================================

print("\n" + "="*70)
print("ADDITIONAL INSIGHTS")
print("="*70)

# Top-level vs Reply comments
if 'is_top_level' in df.columns:
    print("\n Sentiment by Comment Type:")
    for is_top in df['is_top_level'].unique():
        comment_type = "Top-Level Comment" if is_top == 1 else "Reply"
        subset = df[df['is_top_level'] == is_top]
        avg_sentiment = subset['sentiment_score'].mean()
        print(f"  {comment_type}: {avg_sentiment:.3f} (n={len(subset):,})")

# Sentiment over time
if 'comment_created_utc' in df.columns:
    print("\n Analyzing temporal patterns...")
    df['comment_date'] = pd.to_datetime(df['comment_created_utc'])
    df['year_month'] = df['comment_date'].dt.to_period('M')
    
    temporal = df.groupby('year_month')['sentiment_score'].mean()
    print(f"  Data spans from {temporal.index.min()} to {temporal.index.max()}")
    print(f"  Average sentiment over time ranges from {temporal.min():.3f} to {temporal.max():.3f}")

# Comment score correlation
if 'comment_score' in df.columns:
    correlation = df['comment_score'].corr(df['sentiment_score'])
    print(f"\n🔗 Correlation between comment score and sentiment: {correlation:.3f}")

print("\n" + "="*70)
print(" ANALYSIS COMPLETE!")
print("="*70)

print(f"""
 Files created:
   - reddit_data_with_sentiment.csv (your data with sentiment)
   - reddit_sentiment_analysis.png (visualizations)

 Key findings:
   - Total comments analyzed: {len(df):,}
   - Average sentiment: {df['sentiment_score'].mean():.3f}
   - Most common sentiment: {df['sentiment_label'].value_counts().index[0]}
   - Positive comments: {(df['sentiment_label']=='Positive').sum():,} ({(df['sentiment_label']=='Positive').sum()/len(df)*100:.1f}%)
   - Negative comments: {(df['sentiment_label']=='Negative').sum():,} ({(df['sentiment_label']=='Negative').sum()/len(df)*100:.1f}%)


""")

print("="*70)

REDDIT SENTIMENT ANALYSIS

 Loading Reddit data...
 Loaded 25,148 comments

Columns in your data:
['comment_id', 'post_id', 'subreddit', 'author_username', 'comment_body', 'comment_score', 'comment_created_utc', 'parent_id', 'is_top_level', 'post_title', 'post_text', 'post_created_utc', 'post_url']

 Initializing sentiment analyzer...
 Ready to analyze!

 Analyzing 25,148 comments...
This may take a few minutes for large datasets...
  Processed 5,000/25,148 comments...
  Processed 10,000/25,148 comments...
  Processed 15,000/25,148 comments...
  Processed 20,000/25,148 comments...
  Processed 25,000/25,148 comments...
✓ Completed analyzing 25,148 comments!

 Adding sentiment results to dataframe...
 Sentiment scores added!

RESULTS SUMMARY

 Sentiment Distribution:
sentiment_label
Positive    12310
Negative     7288
Neutral      5550
Name: count, dtype: int64

 Sentiment Percentages:
  Positive: 49.0%
  Negative: 29.0%
  Neutral: 22.1%

 Average Sentiment Score: 0.139
   Standard Devia

In [34]:
import pandas as pd

# --- Load data ---
df = pd.read_csv("glassdoor_health_care_reviews.csv")



In [46]:
# --- Preview ---
print("first rows:")
print(df.head())



first rows:
   review_id       company_name    industry                     job_title  \
0          1  Boston Scientific  Healthcare           Process engineer ii   
1          2  Boston Scientific  Healthcare  Ep senior mapping specialist   
2          3  Boston Scientific  Healthcare      Manufacturing technician   
3          4  Boston Scientific  Healthcare             Territory manager   
4          5  Boston Scientific  Healthcare      Senior financial analyst   

  review_date  overall_rating  \
0  2025-11-07             5.0   
1  2025-11-02             2.0   
2  2025-11-03             5.0   
3  2025-11-08             4.0   
4  2025-10-29             2.0   

                                           pros_text  \
0                    Great culture with smart people   
1  Benefits - good health and dental insurance ES...   
2  Great work environment A lot of help from uppe...   
3               Above average pay and great benefits   
4                  Great pay and some solid be

In [45]:
# --- Dimensions ---
print("\nshape (rows, columns):")
print(df.shape)




shape (rows, columns):
(1500, 8)


In [44]:
# --- Variable names ---
print("\nvariable names:")
print(list(df.columns))




variable names:
['review_id', 'company_name', 'industry', 'job_title', 'review_date', 'overall_rating', 'pros_text', 'cons_text']


In [43]:
# --- Structure ---
print("\ndata info:")
print(df.info())




data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   review_id       1500 non-null   int64  
 1   company_name    1500 non-null   object 
 2   industry        1500 non-null   object 
 3   job_title       1500 non-null   object 
 4   review_date     1500 non-null   object 
 5   overall_rating  1500 non-null   float64
 6   pros_text       1500 non-null   object 
 7   cons_text       1500 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 93.9+ KB
None


In [42]:
# --- Summary stats ---
print("\nsummary stats")
print(df.describe(include='all'))




summary stats
          review_id company_name    industry           job_title review_date  \
count   1500.000000         1500        1500                1500        1500   
unique          NaN            6           1                 357         222   
top             NaN       Pfizer  Healthcare  Anonymous employee  2025-10-14   
freq            NaN          281        1500                 152          33   
mean     750.500000          NaN         NaN                 NaN         NaN   
std      433.157015          NaN         NaN                 NaN         NaN   
min        1.000000          NaN         NaN                 NaN         NaN   
25%      375.750000          NaN         NaN                 NaN         NaN   
50%      750.500000          NaN         NaN                 NaN         NaN   
75%     1125.250000          NaN         NaN                 NaN         NaN   
max     1500.000000          NaN         NaN                 NaN         NaN   

        overall_rating  

In [41]:
# --- Missing values ---
print("\nmissing value count:")
print(df.isnull().sum())



missing value count:
review_id         0
company_name      0
industry          0
job_title         0
review_date       0
overall_rating    0
pros_text         0
cons_text         0
dtype: int64


In [48]:
"""
SENTIMENT ANALYSIS FOR GLASSDOOR HEALTHCARE REVIEWS

"""

import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns

print("="*70)
print("GLASSDOOR HEALTHCARE SENTIMENT ANALYSIS")
print("="*70)

# ============================================================================
# STEP 1: LOAD YOUR DATA
# ============================================================================

print("\n Loading Glassdoor Healthcare reviews...")


df = pd.read_csv('glassdoor_health_care_reviews.csv')

print(f"✓ Loaded {len(df):,} reviews")
print(f"\nColumns in your data:")
print(df.columns.tolist())

# ============================================================================
# STEP 2: INITIALIZE SENTIMENT ANALYZER
# ============================================================================

print("\n Initializing sentiment analyzer...")
analyzer = SentimentIntensityAnalyzer()
print(" Ready to analyze!")

# ============================================================================
# STEP 3: ANALYZE SENTIMENT FOR PROS AND CONS
# ============================================================================

print(f"\n Analyzing {len(df):,} reviews...")
print("Analyzing both PROS and CONS sections...")

results = []
for idx, row in df.iterrows():
    # Analyze PROS
    pros_text = row['pros_text']
    if pd.isna(pros_text) or str(pros_text).strip() == "" or str(pros_text) == "N/A":
        pros_score = 0.0
        pros_label = 'Neutral'
    else:
        pros_scores = analyzer.polarity_scores(str(pros_text))
        pros_score = pros_scores['compound']
        pros_label = 'Positive' if pros_score >= 0.05 else ('Negative' if pros_score <= -0.05 else 'Neutral')
    
    # Analyze CONS
    cons_text = row['cons_text']
    if pd.isna(cons_text) or str(cons_text).strip() == "" or str(cons_text) == "N/A":
        cons_score = 0.0
        cons_label = 'Neutral'
    else:
        cons_scores = analyzer.polarity_scores(str(cons_text))
        cons_score = cons_scores['compound']
        cons_label = 'Positive' if cons_score >= 0.05 else ('Negative' if cons_score <= -0.05 else 'Neutral')
    
    # Calculate COMBINED sentiment (average of pros and cons)
    combined_score = (pros_score + cons_score) / 2
    combined_label = 'Positive' if combined_score >= 0.05 else ('Negative' if combined_score <= -0.05 else 'Neutral')
    
    results.append({
        'pros_sentiment_score': pros_score,
        'pros_sentiment_label': pros_label,
        'cons_sentiment_score': cons_score,
        'cons_sentiment_label': cons_label,
        'combined_sentiment_score': combined_score,
        'combined_sentiment_label': combined_label
    })
    
    # Progress indicator
    if (idx + 1) % 500 == 0:
        print(f"  Processed {idx + 1:,}/{len(df):,} reviews...")

print(f" Completed analyzing {len(df):,} reviews!")

# ============================================================================
# STEP 4: ADD RESULTS TO DATAFRAME
# ============================================================================

print("\n Adding sentiment results to dataframe...")

# Convert results to dataframe
sentiment_df = pd.DataFrame(results)

# Add to original dataframe
df = pd.concat([df, sentiment_df], axis=1)

print(" Sentiment scores added!")

# ============================================================================
# STEP 5: SHOW RESULTS SUMMARY
# ============================================================================

print("\n" + "="*70)
print("RESULTS SUMMARY")
print("="*70)

print("\n COMBINED Sentiment Distribution:")
print(df['combined_sentiment_label'].value_counts())

print("\n COMBINED Sentiment Percentages:")
sentiment_pct = df['combined_sentiment_label'].value_counts(normalize=True) * 100
for label, pct in sentiment_pct.items():
    print(f"  {label}: {pct:.1f}%")

print(f"\n Average Sentiment Scores:")
print(f"   PROS:     {df['pros_sentiment_score'].mean():.3f}")
print(f"   CONS:     {df['cons_sentiment_score'].mean():.3f}")
print(f"   COMBINED: {df['combined_sentiment_score'].mean():.3f}")

# ============================================================================
# STEP 6: SENTIMENT BY COMPANY
# ============================================================================

print("\n" + "="*70)
print("SENTIMENT BY COMPANY")
print("="*70)

company_stats = df.groupby('company_name').agg({
    'combined_sentiment_score': 'mean',
    'review_id': 'count',
    'overall_rating': 'mean'
}).round(3)

company_stats.columns = ['Avg Sentiment', 'Review Count', 'Avg Rating']
company_stats = company_stats.sort_values('Avg Sentiment', ascending=False)

print("\n", company_stats)

# ============================================================================
# STEP 7: SHOW EXAMPLES
# ============================================================================

print("\n" + "="*70)
print("examples")
print("="*70)

print("\n Most Positive Reviews:")
top_positive = df.nlargest(3, 'combined_sentiment_score')
for idx, row in top_positive.iterrows():
    print(f"\n  Company: {row['company_name']}")
    print(f"  Job Title: {row['job_title']}")
    print(f"  Score: {row['combined_sentiment_score']:.3f}")
    print(f"  Pros: {str(row['pros_text'])[:100]}...")
    print(f"  Cons: {str(row['cons_text'])[:100]}...")

print("\n\n Most Negative Reviews:")
top_negative = df.nsmallest(3, 'combined_sentiment_score')
for idx, row in top_negative.iterrows():
    print(f"\n  Company: {row['company_name']}")
    print(f"  Job Title: {row['job_title']}")
    print(f"  Score: {row['combined_sentiment_score']:.3f}")
    print(f"  Pros: {str(row['pros_text'])[:100]}...")
    print(f"  Cons: {str(row['cons_text'])[:100]}...")

# ============================================================================
# STEP 8: CREATE VISUALIZATIONS
# ============================================================================

print("\n" + "="*70)
print("CREATING VISUALIZATIONS")
print("="*70)

# Set style
sns.set_style("whitegrid")

# Create figure with multiple subplots
fig = plt.figure(figsize=(18, 12))
fig.suptitle('Glassdoor Healthcare Sentiment Analysis', fontsize=16, fontweight='bold')

# Chart 1: Combined Sentiment Distribution
ax1 = plt.subplot(2, 3, 1)
sentiment_counts = df['combined_sentiment_label'].value_counts()
colors = {'Positive': '#2ecc71', 'Neutral': '#95a5a6', 'Negative': '#e74c3c'}
bar_colors = [colors.get(label, '#3498db') for label in sentiment_counts.index]

ax1.bar(sentiment_counts.index, sentiment_counts.values, color=bar_colors)
ax1.set_title('Combined Sentiment Distribution', fontsize=12, fontweight='bold')
ax1.set_ylabel('Number of Reviews')
ax1.set_xlabel('Sentiment')

# Add count labels
for i, v in enumerate(sentiment_counts.values):
    ax1.text(i, v, f'{v:,}\n({v/len(df)*100:.1f}%)', 
            ha='center', va='bottom', fontsize=9, fontweight='bold')

# Chart 2: Pros vs Cons Sentiment
ax2 = plt.subplot(2, 3, 2)
comparison_data = pd.DataFrame({
    'Pros': [df['pros_sentiment_score'].mean()],
    'Cons': [df['cons_sentiment_score'].mean()],
    'Combined': [df['combined_sentiment_score'].mean()]
})
comparison_data.T.plot(kind='bar', ax=ax2, legend=False, color=['#2ecc71', '#e74c3c', '#3498db'])
ax2.set_title('Average Sentiment: Pros vs Cons', fontsize=12, fontweight='bold')
ax2.set_ylabel('Average Sentiment Score')
ax2.set_xticklabels(['Pros', 'Cons', 'Combined'], rotation=0)
ax2.axhline(0, color='black', linestyle='-', linewidth=0.8)
ax2.grid(alpha=0.3, axis='y')

# Chart 3: Sentiment Score Distribution
ax3 = plt.subplot(2, 3, 3)
ax3.hist(df['combined_sentiment_score'], bins=40, color='steelblue', alpha=0.7, edgecolor='black')
ax3.axvline(df['combined_sentiment_score'].mean(), color='red', linestyle='--', 
           linewidth=2, label=f'Mean: {df["combined_sentiment_score"].mean():.3f}')
ax3.set_title('Combined Sentiment Score Distribution', fontsize=12, fontweight='bold')
ax3.set_xlabel('Sentiment Score')
ax3.set_ylabel('Frequency')
ax3.legend()
ax3.grid(alpha=0.3)

# Chart 4: Sentiment by Company
ax4 = plt.subplot(2, 3, 4)
company_sentiment = df.groupby('company_name')['combined_sentiment_score'].mean().sort_values()
colors_comp = ['#e74c3c' if x < -0.05 else '#2ecc71' if x > 0.05 else '#95a5a6' 
               for x in company_sentiment.values]

ax4.barh(company_sentiment.index, company_sentiment.values, color=colors_comp)
ax4.set_title('Average Sentiment by Company', fontsize=12, fontweight='bold')
ax4.set_xlabel('Average Sentiment Score')
ax4.axvline(0, color='black', linestyle='-', linewidth=0.8)
ax4.grid(alpha=0.3, axis='x')

# Add review counts
for i, (idx, value) in enumerate(company_sentiment.items()):
    count = len(df[df['company_name'] == idx])
    ax4.text(value, i, f' n={count}', va='center', fontsize=8)

# Chart 5: Rating vs Sentiment Correlation
ax5 = plt.subplot(2, 3, 5)
ax5.scatter(df['overall_rating'], df['combined_sentiment_score'], alpha=0.3, s=20)
ax5.set_title('Overall Rating vs Sentiment Score', fontsize=12, fontweight='bold')
ax5.set_xlabel('Overall Rating (Stars)')
ax5.set_ylabel('Sentiment Score')
ax5.grid(alpha=0.3)

# Add correlation
correlation = df['overall_rating'].corr(df['combined_sentiment_score'])
ax5.text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
        transform=ax5.transAxes, fontsize=10, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Chart 6: Top/Bottom Companies
ax6 = plt.subplot(2, 3, 6)
top_3 = company_sentiment.tail(3)
bottom_3 = company_sentiment.head(3)
combined_top_bottom = pd.concat([bottom_3, top_3])

colors_tb = ['#e74c3c']*len(bottom_3) + ['#2ecc71']*len(top_3)
ax6.barh(range(len(combined_top_bottom)), combined_top_bottom.values, color=colors_tb)
ax6.set_yticks(range(len(combined_top_bottom)))
ax6.set_yticklabels(combined_top_bottom.index, fontsize=9)
ax6.set_title('Top 3 & Bottom 3 Companies', fontsize=12, fontweight='bold')
ax6.set_xlabel('Average Sentiment Score')
ax6.axvline(0, color='black', linestyle='-', linewidth=0.8)
ax6.grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('glassdoor_healthcare_sentiment_analysis.png', dpi=300, bbox_inches='tight')
print(" Saved visualization: glassdoor_healthcare_sentiment_analysis.png")
plt.close()

# ============================================================================
# STEP 9: SAVE RESULTS TO CSV
# ============================================================================

print("\n" + "="*70)
print("SAVING RESULTS")
print("="*70)

output_file = 'glassdoor_health_care_reviews_with_sentiment.csv'
df.to_csv(output_file, index=False)

print(f"✓ Saved results to: {output_file}")
print(f"  Total rows: {len(df):,}")
print(f"  Total columns: {len(df.columns)}")
print(f"\nNew columns added:")
print(f"  - pros_sentiment_score")
print(f"  - pros_sentiment_label")
print(f"  - cons_sentiment_score")
print(f"  - cons_sentiment_label")
print(f"  - combined_sentiment_score (MAIN METRIC)")
print(f"  - combined_sentiment_label")

# ============================================================================
# STEP 10: ADDITIONAL INSIGHTS
# ============================================================================

print("\n" + "="*70)
print("ADDITIONAL INSIGHTS")
print("="*70)

# Correlation between rating and sentiment
correlation = df['overall_rating'].corr(df['combined_sentiment_score'])
print(f"\n Correlation between star rating and sentiment: {correlation:.3f}")

# Sentiment by job title (top 10)
print("\n Sentiment by Job Title (Top 10 most common):")
job_sentiment = df.groupby('job_title').agg({
    'combined_sentiment_score': 'mean',
    'review_id': 'count'
})
job_sentiment.columns = ['Avg Sentiment', 'Count']
job_sentiment = job_sentiment[job_sentiment['Count'] >= 3]  # At least 3 reviews
job_sentiment = job_sentiment.sort_values('Avg Sentiment', ascending=False).head(10)
print(job_sentiment.round(3))

# Time analysis (if date is available)
if 'review_date' in df.columns:
    try:
        df['review_date'] = pd.to_datetime(df['review_date'])
        df['year_month'] = df['review_date'].dt.to_period('M')
        
        temporal = df.groupby('year_month')['combined_sentiment_score'].mean()
        print(f"\n Temporal Analysis:")
        print(f"  Data spans from {temporal.index.min()} to {temporal.index.max()}")
        print(f"  Sentiment ranges from {temporal.min():.3f} to {temporal.max():.3f}")
    except:
        print("\n Could not parse review dates for temporal analysis")

print("\n" + "="*70)
print(" ANALYSIS COMPLETE!")
print("="*70)

print(f"""
 Files created:
   - glassdoor_health_care_reviews_with_sentiment.csv (data with sentiment)
   - glassdoor_healthcare_sentiment_analysis.png (6 charts)

 Key findings:
   - Total reviews analyzed: {len(df):,}
   - Average COMBINED sentiment: {df['combined_sentiment_score'].mean():.3f}
   - Average PROS sentiment: {df['pros_sentiment_score'].mean():.3f}
   - Average CONS sentiment: {df['cons_sentiment_score'].mean():.3f}
   - Most positive company: {company_sentiment.idxmax()} ({company_sentiment.max():.3f})
   - Most negative company: {company_sentiment.idxmin()} ({company_sentiment.min():.3f})
   - Positive reviews: {(df['combined_sentiment_label']=='Positive').sum():,} ({(df['combined_sentiment_label']=='Positive').sum()/len(df)*100:.1f}%)
   - Negative reviews: {(df['combined_sentiment_label']=='Negative').sum():,} ({(df['combined_sentiment_label']=='Negative').sum()/len(df)*100:.1f}%)


""")

print("="*70)

GLASSDOOR HEALTHCARE SENTIMENT ANALYSIS

 Loading Glassdoor Healthcare reviews...
✓ Loaded 1,500 reviews

Columns in your data:
['review_id', 'company_name', 'industry', 'job_title', 'review_date', 'overall_rating', 'pros_text', 'cons_text']

 Initializing sentiment analyzer...
 Ready to analyze!

 Analyzing 1,500 reviews...
Analyzing both PROS and CONS sections...
  Processed 500/1,500 reviews...
  Processed 1,000/1,500 reviews...
  Processed 1,500/1,500 reviews...
 Completed analyzing 1,500 reviews!

 Adding sentiment results to dataframe...
 Sentiment scores added!

RESULTS SUMMARY

 COMBINED Sentiment Distribution:
combined_sentiment_label
Positive    1268
Negative     122
Neutral      110
Name: count, dtype: int64

 COMBINED Sentiment Percentages:
  Positive: 84.5%
  Negative: 8.1%
  Neutral: 7.3%

 Average Sentiment Scores:
   PROS:     0.623
   CONS:     0.016
   COMBINED: 0.320

SENTIMENT BY COMPANY

                    Avg Sentiment  Review Count  Avg Rating
company_name      